<a href="https://colab.research.google.com/github/inaciotbueno/rapidfem4d/blob/main/RapidFEM4D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Initialize Earth Engine**

In [ ]:
try:
  import geemap
except ModuleNotFoundError:
  !pip install geemap -q

import ee
import pandas as pd
import numpy as np
import random
import re

from google.colab import drive
from geemap import ml
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

try:
  my_dir
except NameError:
  drive.mount('/content/drive/')
  my_dir = '/content/drive/My Drive'

try:
  ee.Initialize()
except:
  ee.Authenticate()
  ee.Initialize(project='ee-inaciotbueno')

clear_output()

# **2. Define User**

In [ ]:
# Define User Project (Replace with actual EE user project)
USER_PROJECT = "ee-inaciotbueno"

# Define where AGBD models will be stored
MODEL_DIR = f"projects/{USER_PROJECT}/assets/rapidfem4d/agbd/model"

# Define where AGBD maps will be stored
IMG_DIR = f"projects/{USER_PROJECT}/assets/rapidfem4d/agbd/image"

# Define where output files will be saved in Google drive, be sure the folder exists
DRIVE_DIR = '/content/drive/My Drive/rapidfem4d'

# Function to check and create all directories in a given asset path
def ensure_full_asset_path(asset_path):
    base_path = f"projects/{USER_PROJECT}/assets"
    relative_path = asset_path.replace(base_path + "/", "")  # Remove base path from full path

    path_parts = relative_path.split('/')
    current_path = base_path  # Start checking from base path

    for part in path_parts:
        current_path = f"{current_path}/{part}"
        try:
            ee.data.getAsset(current_path)
        except ee.EEException as e:
            if 'does not exist' in str(e):
                ee.data.createAsset({'type': 'Folder'}, current_path)

ensure_full_asset_path(IMG_DIR)
ensure_full_asset_path(MODEL_DIR)

# **3. User control inputs**


In [ ]:
# Define year and response variable
year = '2021'

# Define study area
aoi = ee.FeatureCollection('projects/ee-inaciotbueno/assets/RapidFEM4D/boundary_Ian')

# Define GEDI and HLS time periods
start_date_gedi, end_date_gedi = year+'-04-01', year+'-04-30'
start_date_img, end_date_img = year+'-01-01', year+'-07-30'

# Load pre-existing CSV data for model training if export is not required
try:
  df = pd.read_csv(DRIVE_DIR+'/gedi_samples.csv')
  df['system:index'] = range(len(df))
  exportCSV = 'No'
  print('CSV already exists')

except FileNotFoundError:
  exportCSV = 'Yes'
  print('CSV will be created')

# Define model parameters
rfModel = 'No'  # Use Random Forest Model: Yes | No
nRuns = 5 # Number os model iterations and maps
nTrees = 200  # Number of trees in Random Forest
nSamples = 1000  # Number of samples for model training

CSV already exists


# **4. Define functions**

In [ ]:
def l4a_quality_mask(image):
  filter1 = image.updateMask(image.select('l4_quality_flag').eq(1))
  filter2 = filter1.updateMask(filter1.select('degrade_flag').eq(0))
  return filter2

def full_power(image):
  return (image.updateMask(image.select('beam').gte(4)))

def night(image):
  return image.updateMask(image.select('solar_elevation').lte(0))

def cloud_mask(image):
  qa = image.select(['Fmask']);
  cloudmask = qa.bitwiseAnd(1 << 1).Or(qa.bitwiseAnd(1 << 2)).Or(qa.bitwiseAnd(1 << 3))
  return image.updateMask(cloudmask.Not())

def water_mask(image):
  band5 = image.select('B5')
  mask = band5.gte(0.2)
  return image.updateMask(mask)

def scale_image(image, factor):
    return image.multiply(factor).int()

def load_csv_or_empty(path):
    try:
        return pd.read_csv(path, index_col=[0])
    except FileNotFoundError:
        return pd.DataFrame()

def classifyRaster(image, threshold):
  classes = ee.Image(0).rename('vegetation')
  classes = classes.where(image.select('trees').gt(threshold), 1)
  classes = classes.where(image.select('shrub_and_scrub').gt(threshold), 2)
  classes = classes.where(image.select('grass').gt(threshold), 3)
  classes = classes.where(image.select('flooded_vegetation').gt(threshold), 4)
  return ee.Image.cat([image, classes])

# **5. Create variables**

In [ ]:
# Vegetation mask for gedi footprint selection
dyn_world = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")\
  .filterBounds(aoi)\
  .select('trees')\
  .mosaic()\
  .gt(0.4)

def invBuffer(image, size):
  """ Apply an inverse buffer to mask out areas near edges """
  local_scale = ee.Image.pixelArea().sqrt()
  buffer = image.mask().Not().fastDistanceTransform(size, 'pixels').sqrt().multiply(local_scale)
  return image.updateMask(buffer.gt(size))

vegMask = invBuffer(dyn_world, 100)

# Vegetation mask for final map clipping
dyn_world2 = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")\
  .filterBounds(aoi)\
  .select(['trees','shrub_and_scrub','grass','flooded_vegetation'])\
  .mosaic()

vegMask2 = classifyRaster(dyn_world2, 0.1).select('vegetation')
vegMask2 = vegMask2.updateMask(vegMask2)

# Import GED L4A
gedi_l4a = ee.ImageCollection("LARSE/GEDI/GEDI04_A_002_MONTHLY")\
  .filterBounds(aoi)\
  .filterDate(start_date_gedi, end_date_gedi)\
  .map(l4a_quality_mask)\
  .map(full_power)\
  .select('agbd')\
  .mosaic().int()

# Import Harmonized Landsat Sentinel
hls = ee.ImageCollection("NASA/HLS/HLSL30/v002")\
  .filterBounds(aoi)\
  .filterDate(start_date_img, end_date_img)\
  .filter(ee.Filter.lt("CLOUD_COVERAGE", 30))\
  .map(cloud_mask)\
  .map(water_mask)\
  .median()

hls = scale_image(hls, 10000)\
  .select(['B2', 'B3', 'B4', 'B5', 'B6','B7'], ['blue', 'green','red', 'nir', 'swir1', 'swir2'])

# Import Sentinel 1C
s1c = ee.ImageCollection("COPERNICUS/S1_GRD")\
  .filterBounds(aoi)\
  .filterDate(start_date_img, end_date_img)\
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
  .filter(ee.Filter.eq('instrumentMode', 'IW'))\
  .map(lambda img: scale_image(img, 100))\
  .sort('system:time_start', False)\
  .reduce(ee.Reducer.firstNonNull())\
  .select(['VV_first', 'VH_first'], ['vv', 'vh']).int()

# Terrain properties
elevation = ee.Image("NASA/NASADEM_HGT/001").select('elevation').int()
slope = ee.Terrain.slope(elevation).multiply(1000).int()
aspect = ee.Terrain.aspect(elevation).multiply(10).int16()

# Normalized Difference Vegetation Index
ndvi = hls.normalizedDifference(['nir', 'red']).select(['nd'],['ndvi']).multiply(10000).int();

# Kernel normalized difference vegetation index
def addKNDVI(im):
    D2 = im.select('nir').subtract(im.select('red')).pow(2).rename('d2')
    gamma = ee.Number(4e6).multiply(-2.0)
    k = D2.divide(gamma).exp()
    kndvi = ee.Image.constant(1).subtract(k).divide(ee.Image.constant(1).add(k)).rename('kndvi')
    return im.addBands(kndvi).multiply(1000).int()
kndvi = addKNDVI(hls).select('kndvi')

# Enhanced Vegetation Index
evi = hls.expression('evi = 2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
  {'nir': hls.select('nir'),'red': hls.select('red'),'blue': hls.select('blue')}).multiply(10000).int()

# Soil-Adjusted Vegetation Index
savi = hls.expression('savi = 1.5 * (nir - red) / (nir + red + 0.5)',
  {'nir': hls.select('nir'),'red': hls.select('red')}).multiply(10000).int();

# Modified Soil-Adjusted Vegetation Index
def getMSAVI2(im):
    p1 = (im.select('red').multiply(2).add(1)).pow(2)
    p2 = (im.select('red').subtract(im.select('nir'))).multiply(8)
    p3 = (im.select('red').multiply(2).add(1))
    return im.addBands((p3.subtract((p1.subtract(p2).sqrt())).divide(2).rename('msavi2')))
msavi2 = getMSAVI2(hls).select('msavi2').multiply(10000).int()

# Simple Ratio Index
sri = hls.expression('sri = nir / red',
  {'nir': hls.select('nir'),'red': hls.select('red')}).multiply(1000).int()

# Normalized Difference Water Index
ndwi = hls.normalizedDifference(['green', 'nir']).select(['nd'],['ndwi']).multiply(10000).int()

# Green Chlorophyll Index
gci = hls.expression('gci = (nir / green) - 1',
  {'nir': hls.select('nir'),'green': hls.select('green')}).multiply(1000).int()

wdrvi = hls.expression('wdrvi = ((0.1 * nir) - red) / ((0.1 * nir) + red)',
  {'nir': hls.select('nir'),'red': hls.select('red')}).multiply(10000).int()

# Global Vegetation Moisture Index
gvmi = hls.expression('gvmi = ((nir + 0.1) - (swir1 + 0.02)) / ((nir + 0.1) + (swir1 + 0.02))',
  {'swir1': hls.select('swir1'),'nir': hls.select('nir')}).multiply(10000).int()

# Chlorophyll vegetation index
def getCVI(im):
    p1 = im.select('nir');
    p2 = im.select('red');
    p3 = im.select('green').pow(2);
    p4 = p2.divide(p3);
    return p1.multiply(p4).rename('cvi')
cvi = getCVI(hls).select('cvi').multiply(1000).int()

# Clay minerals ratio
cmr = hls.expression('cmr = swir1 / swir2',
  {'swir1': hls.select('swir1'),'swir2': hls.select('swir2')}).multiply(1000).int()

# Radar Vegetation Index
rvi = s1c.expression('rvi = sqrt(vv/(vv+vh))*(vv/vh)',
  {'vv': s1c.select('vv'),'vh': s1c.select('vh')}).multiply(10000).int()

# Radar co-polarized 1
copol = s1c.expression('copol = vv / vh',
  {'vv': s1c.select('vv'),'vh': s1c.select('vh')}).multiply(10000).int()

# Radar co-polarized 2
copol2 = s1c.expression('copol2 = (vv-vh)/(vv+vh)',
  {'vv': s1c.select('vv'),'vh': s1c.select('vh')}).multiply(10000).int()

# Radar co-polarized 3
copol3 = s1c.expression('copol3 = vh / vv',
  {'vv': s1c.select('vv'),'vh': s1c.select('vh')}).multiply(10000).int()

x_coord = ee.Image.pixelLonLat().select('longitude')
y_coord = ee.Image.pixelLonLat().select('latitude')

# Spatial transformations and image stack
stackS2 = ee.Image.cat([ndvi,kndvi,evi,savi,msavi2,sri,ndwi,gci,wdrvi,gvmi,cvi,cmr,hls])
stackS1 = ee.Image.cat([rvi,copol,copol2,copol3,s1c])
stackDem = ee.Image.cat([elevation,slope,aspect])

# 3x3 Mean
kernel = ee.Kernel.fixed(3, 3, [[1, 1, 1], [1, 1, 1], [1, 1, 1]], 3, 3, False)
stackS2_mn = stackS2.reduceNeighborhood(ee.Reducer.mean(), kernel)
stackS1_mn = stackS1.reduceNeighborhood(ee.Reducer.mean(), kernel)
stackDem_mn = stackDem.reduceNeighborhood(ee.Reducer.mean(), kernel)

# 3x3 Maximum
stackS2_mx = stackS2.reduceNeighborhood(ee.Reducer.max(), kernel)
stackS1_mx = stackS1.reduceNeighborhood(ee.Reducer.max(), kernel)
stackDem_mx = stackDem.reduceNeighborhood(ee.Reducer.max(), kernel)

# 3x3 Minimum
stackS2_mi = stackS2.reduceNeighborhood(ee.Reducer.min(), kernel)
stackS1_mi = stackS1.reduceNeighborhood(ee.Reducer.min(), kernel)
stackDem_mi = stackDem.reduceNeighborhood(ee.Reducer.min(), kernel)

# 3x3 Standard deviation
stackS2_std = stackS2.reduceNeighborhood(ee.Reducer.stdDev(), kernel)
stackS1_std = stackS1.reduceNeighborhood(ee.Reducer.stdDev(), kernel)
stackDem_std = stackDem.reduceNeighborhood(ee.Reducer.stdDev(), kernel)

# GLCM
glcm = hls.glcmTexture(**{'size': 3})

stack = ee.Image.cat([
    gedi_l4a,
    stackS2, stackS1,stackDem,
    stackS2_mn, stackS1_mn, stackDem_mn,
    stackS2_mx, stackS1_mx, stackDem_mx,
    stackS2_mi, stackS1_mi, stackDem_mi,
    stackS2_std, stackS1_std, stackDem_std,
    glcm,
    x_coord, y_coord
])

# **6. Exporting GEDI to .csv**

In [ ]:
if (exportCSV == 'Yes'):

  # Select and mask image bands
  agbdImg = stack.select('agbd')

  # Sample points from the processed image
  gediSamples = agbdImg.sample(**{
      'region': aoi,
      'scale': 25,
      'factor': 0.1,
      'geometries': True,
      'tileScale': 16
  })

  # Reduce regions to get values for each Gedi pixel
  gediZonal = stack.reduceRegions(**{
    'collection': gediSamples,
    'reducer': ee.Reducer.firstNonNull(),
    'scale': 30,
    'tileScale': 16
  })

  # Configure the export task for CSV export
  task_config = {
      'description': 'gedi_samples',
      'folder': DRIVE_DIR.split('/')[-1],
      'fileFormat': 'CSV'
  }
  # Start the export task
  task = ee.batch.Export.table.toDrive(gediZonal, **task_config)
  task.start()

  raise Exception("Execution stopped. Wait for the submitted task to be completed.")

# **7. AGBD modeling**

In [ ]:
new_df = df.dropna().reset_index(drop=True)
feature_names = [col for col in new_df.columns if col not in ['agbd', '.geo', 'system:index', 'year']]


# Run Random Forest model if enabled
if rfModel == 'Yes':
    score = pd.DataFrame()
    bootstrap = []

    # Load existing importance, metrics, and prediction data
    def load_csv_or_empty(path):
        try:
            return pd.read_csv(path, index_col=[0])
        except FileNotFoundError:
            return pd.DataFrame()

    metrics = load_csv_or_empty(DRIVE_DIR+'/metrics_agbd.csv')
    predictions = load_csv_or_empty(DRIVE_DIR+'/predictions_agbd.csv')

    # Loop through multiple runs
    for i in range(0,nRuns):

      if (i < 9):
        filenumber = '00'+str(i+1)
      elif ((i >= 9) and (i < 99)):
        filenumber = '0'+str(i+1)
      else:
        filenumber = str(i+1)

      assetPath = MODEL_DIR+'/agbd_model_'+filenumber

      try:
        assetFile = ee.data.getAsset(assetPath)
      except :

        # Prepare training and testing data
        new_df_sample = new_df.sample(1000, random_state = random.randint(1, 10000))
        train_df, test_df = train_test_split(new_df_sample, test_size=0.3, random_state=random.randint(1, 10000))
        X_train = train_df[feature_names]
        y_train = train_df[['agbd']]
        X_test = test_df[feature_names]
        y_test = test_df[['agbd']]
        y_train = np.ravel(y_train)

        # Train Random Forest model
        rf = RandomForestRegressor(n_estimators=nTrees)
        selector = SelectFromModel(estimator=rf, max_features=None)
        X_train_new = selector.fit_transform(X_train, y_train)
        X_test_new = selector.transform(X_test)
        rf.fit(X_train_new, y_train)
        y_pred = rf.predict(X_test_new)

        selected_feature_indices = selector.get_support(indices=True)
        selected_feature_names = [feature_names[i] for i in selected_feature_indices]

        # Print results of each run
        print(50*'--',"\nRepeat", i+1, "- ", 'agbd', "- Training set size:", len(X_train_new),"- Test set size::", len(X_test_new))
        obs = y_test['agbd']
        obs = pd.to_numeric(obs, errors='coerce')
        pred = pd.Series(y_pred, index = X_test.index).astype('object')
        pred = pd.to_numeric(pred, errors='coerce')

        abs_rmse = round(np.sqrt(sum((pred-obs) ** 2 ) / len(pred)), 2)
        abs_bias = round(np.mean(pred) - np.mean(obs), 4)
        rel_bias = round(abs_bias / np.mean(obs) * 100, 4)
        abs_rmse = round(np.sqrt(sum((pred-obs) ** 2 ) / len(pred)), 2)
        rel_rmse = round(abs_rmse / np.mean(obs) * 100, 2)
        r2 = np.corrcoef(obs, pred) ** 2
        r2 = round(r2[0,1], 4)

        print('BIAS: ', abs_bias)
        print('BIAS (%): ', rel_bias)
        print('RMSE: ', abs_rmse)
        print('RMSE %: ', rel_rmse)
        print('R2: ', r2)

        score.loc[i, 'bootstrap'] = i+1
        score.loc[i, 'BIAS'] = abs_bias
        score.loc[i, 'BIAS(%)'] = rel_bias
        score.loc[i, 'RMSE'] = abs_rmse
        score.loc[i, 'RMSE(%)'] = rel_rmse
        score.loc[i, 'R2'] = r2

        # Save outputs and export model trees as gee asset
        metrics = pd.concat([metrics , score], axis=0)
        score = pd.DataFrame()

        obs.columns = ['obs']
        obs = obs.reset_index(drop=True)
        pred = pred.to_frame()
        pred.columns = ['pred']
        pred = pred.reset_index(drop=True)
        predictions = pd.concat([predictions , obs, pred], axis=1)

        trees = ml.rf_to_strings(rf, selected_feature_names, output_mode='REGRESSION')

        for j in range(nTrees):
          a = trees[j].split('\n', 1)[0]
          lines = trees[j].split('\n')
          lines = lines[1:]
          for line in lines:
            line = line.replace('.',',')
            if (line.find('*') > 0):
              b = re.sub(',.*? ', ' ', line)
            else:
              b = re.sub(',.*? ', ' ', line)
              b = re.sub('\,.*', '', b)
            c = '\n'.join([a, b])
            a = c
          trees[j] = a
          trees[j] = trees[j].replace(' <= ', '<=')
          trees[j] = trees[j].replace(' < ', '<')
          trees[j] = trees[j].replace(' => ', '=>')
          trees[j] = trees[j].replace(' > ', '>')

        ee_classifier = ml.strings_to_classifier(trees)
        ml.export_trees_to_fc(trees, assetPath, description='agbd_model_'+filenumber)

        metrics.to_csv(DRIVE_DIR+'/metrics_agbd.csv')
        predictions.to_csv(DRIVE_DIR+'/predictions_agbd.csv')

# **8. AGBD Upscaling**

In [ ]:
# Extract the models saved
try:
  response = ee.data.listAssets({'parent': MODEL_DIR})
  asset_list = response.get('assets', [])
except:
  asset_list = []

# Load the model and classify the image
for asset in asset_list:
  model_name = asset['name'].split('/')[-1]
  filenumber = model_name.replace("agbd_model_", "")
  img_path = f"{IMG_DIR}/agbd_{year}_{filenumber}"

  try:
      ee.data.getAsset(img_path)
  except:
      rf_fc = ee.FeatureCollection(asset['name'])
      classifier = ml.fc_to_classifier(rf_fc)
      classified = stack.select(feature_names).classify(classifier).updateMask(vegMask2)

      task_config1 = {
          'description': f"agbd_{year}_{filenumber}",
          'assetId': img_path,
          'region': aoi.bounds(),
          'scale': 30,
          'maxPixels': 1e13
      }
      task = ee.batch.Export.image.toAsset(classified, **task_config1)
      task.start()